In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:90%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:11pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:10pt;padding:5px;}
table.dataframe{font-size:10px;}
</style>
"""))

In [2]:
import warnings
import os
import logging
# 경고 제거
warnings.filterwarnings('ignore')

# transformers 로깅 레벨 조정
logging.getLogger("transformers").setLevel(logging.ERROR)

# Hugging Face symlink 경고 제거
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# ↓ 아래는 쓰면 안 돼
# from transformers import pipeline, logging as hf_logging
# hf_logging.set_verbosity_error()

<b><font size="6" color="red">Ch.1 허깅페이스</font><b>

- pipeline() 함수 이용 : 모델을 다운로드 받아 local에서
- Inference API 이용 : 모델의 결과를 server에서

    - raw text → tokenizer → model → [0.03, 0.55, 0.xx ...] logits 값으로 prediction 결과 출력
    ```
    허깅페이스 transformers에서 지원하는 task
    sentiment-analysis : text-classification의 별칭 (주로 긍정/부정 감정분석용)
    text-classification : 뉴스 분류, 스팸 필터링 등 일반적인 문장 분류
    zero-shot-classification : 별도 학습 없이 후보 레이블(예: 정치, 경제)만 주고 분류
    token-classification : 개체명 인식(NER), 품사 태깅 등 단어 단위 레이블링
    ner : token-classification의 별칭 (Named Entity Recognition)
    fill-mask : 문장 중간의 빈칸([MASK]) 채우기 (BERT류 학습 방식)
    text-generation : 텍스트 생성, 이어 쓰기 (GPT류 Causal LM 모델)
    text2text-generation : 입력과 출력이 모두 텍스트인 작업 (번역, 요약, T5/BART류)
    translation : 번역 (주로 translation_en_to_fr 처럼 언어 지정이 필요할 때 많음)
    summarization : 긴 텍스트 요약
    question-answering : 지문(Context) 내에서 정답을 찾아 대답 (Extractive QA)
    feature-extraction : 문장을 벡터(임베딩)로 변환 (유사도 측정, 검색용)
    image-classification : 이미지 분류 (개/고양이 분류 등)
    image-to-text : 이미지 캡셔닝 (그림을 보고 텍스트로 묘사)
    automatic-speech-recognition : 음성 인식 (Audio to Text)
    ```

# 1. 텍스트 기반 감정 분석 (긍정/부정)
- C:\Users\Admin\.cache\huggingface\hub 모델 다운로드

In [3]:
from transformers import pipeline
classifier = pipeline(
    task="sentiment-analysis",
    #model='distilbert/distilbert-base-uncased-finetuned-sst-2-english', # 기본 모델
)
#https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

In [5]:
classifier = pipeline(
    task="text-classification",
    model='distilbert/distilbert-base-uncased-finetuned-sst-2-english',
)
# 감정 분석 시 내용이 많으면 list로
classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much"
])

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9995144605636597}]

In [6]:
classifier(["이 영화 정말 나쁘지 않았어요. 감동적이고 악역이 무서워",
           "This film was not bad at all. It's touching and the villan's scary."])

[{'label': 'POSITIVE', 'score': 0.8527063727378845},
 {'label': 'POSITIVE', 'score': 0.9997490048408508}]

In [7]:
classifier("이 물건 정말 사고 싶어요")

[{'label': 'POSITIVE', 'score': 0.8577604293823242}]

In [9]:
classifier(["I like you", "I hat you", "나 너가 싫어", "힘들어요"])

[{'label': 'LABEL_1', 'score': 0.9252550005912781},
 {'label': 'LABEL_1', 'score': 0.8665552735328674},
 {'label': 'LABEL_0', 'score': 0.8790438175201416},
 {'label': 'LABEL_0', 'score': 0.9140233397483826}]

In [11]:
classifier = pipeline(task="text-classification", model="matthewburke/korean_sentiment")
texts = ["이게 뭐야", '그러지 마', '피곤하네', '공원에 가고 싶은 날씨야']
result = classifier(texts)
result

Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.9536595940589905},
 {'label': 'LABEL_0', 'score': 0.7787692546844482},
 {'label': 'LABEL_0', 'score': 0.9584119319915771},
 {'label': 'LABEL_1', 'score': 0.9237558841705322}]

In [13]:
for text, result in zip(texts, classifier(texts)):
    label = "긍정" if result['label']=='LABEL_1' else "부정"
    print(f"{text} → {label} : {result['score']:.4f}")

이게 뭐야 → 부정 : 0.9537
그러지 마 → 부정 : 0.7788
피곤하네 → 부정 : 0.9584
공원에 가고 싶은 날씨야 → 긍정 : 0.9238


# 2. 제로샷 분류
- 기계 학습 및 자연어 처리에서 개별 작업에 대한 특정 교육 없이 작업을 수행할 수 있음
    - 비지도학습으로 학습돼 있음

In [14]:
classifier = pipeline(
    task="zero-shot-classification", 
#     model="facebook/bart-large-mnli"
)
classifier(
    "I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=['urgent', 'not urgent', 'phone', 'computer', 'tablet']
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227580070495605,
  0.45814019441604614,
  0.0142647260800004,
  0.0026850001886487007,
  0.002152054337784648]}

In [15]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'one day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.9938651919364929, 0.0032737581059336662, 0.0028610501904040575]}

# 3. Text 생성

In [24]:
from transformers import set_seed
# set_seed=2
generation = pipeline(
    task = "text-generation", 
    model="openai-community/gpt2"
                     )
# 텍스트 생성 gpt3부터는 허깅페이스에 (아직) 없음
generation(
     "In this course, we will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
          ) 

Device set to use cpu


[{'generated_text': "In this course, we will teach you how to create a database for your application.\n\nDatabase Design\n\nDatabase Design helps you to create your application with a very simple concept.\n\nAn object is a collection of elements. A collection of elements comes in two forms:\n\nA list of elements. A list of elements is an array of cells with a single column\n\nA list of cells is an array of cells with a single column A list of cells with a single column\n\nA cell is a collection of cells with a single column. An object is a collection of cells with a single column\n\nA list of cells with a single column. An object is a collection of cells with a single column.\n\nThe collection is then stored in a table called the database.\n\nLet's take a look at how to create a table.\n\nThis example uses the list function in SQL Server.\n\n// Create a table and create a data object that will be used on the database. public class DatabaseTable { public ICollectionTable bTable; public 

In [31]:
generation = pipeline("text-generation", "skt/kogpt2-base-v2")
result = generation(
    "어려서부터 우리 집은 가난했었고, 남들 다 하는 외식 한 번 한 적이 없었죠. 어머니가 일터에 나가 집에 안 계시면,",
    pad_token_id=generation.tokenizer.eos_token_id,
    max_new_tokens = 100, # 생성할 최대 길이(생성할 토큰 수)
    num_return_sequences=1, # 생성할 문장 갯수
    do_sample=True, # 다양한 샘플 사용
    top_k=50, # 샘플링 (확률 높은 상위) 50개 토큰만 사용
    top_p=0.95, # 확률이 높은 순서대로 95%가 될 때까지의 단어들만 후보로 사용
    temperature=1.2, # 창의성 조절(낮을수록 보수적)
    no_repeat_ngram_size=2, #반복 방지
)
print(result[0]['generated_text'])

Device set to use cpu


어려서부터 우리 집은 가난했었고, 남들 다 하는 외식 한 번 한 적이 없었죠. 어머니가 일터에 나가 집에 안 계시면, 나는 어머니께 혼이 나서 일을 할 수가 없었어요."
"그게 얼마나 중요한 얘긴지요?"
정철은 자기 말이 끝나기를 기다렸다.
"물론 어머니는 당신을 보고 웃을 줄은 몰랐겠지요. 아버지께서 말씀하셨듯이 우리 집이 가난하다면, 어머니는 자기 집에 못 들어오는 한이 있더라도 제게 안경을 끼라고 했을 거예요. 그렇지만 어머니의 얼굴에 웃음꽃이 지고, 당신 어머니가 당신은 어머니께서 안 좋아하시겠지만 이 세상 사람들은 제가 당신에게


# 4. 마스크(빈 칸) 채우기

In [33]:
unmasker = pipeline(
    task='fill-mask', 
    model='distilbert/distilroberta-base'
)
unmasker("I'm going to the hospital and meet a <mask>")

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.2635924518108368,
  'token': 3299,
  'token_str': ' doctor',
  'sequence': "I'm going to the hospital and meet a doctor"},
 {'score': 0.0871344655752182,
  'token': 9008,
  'token_str': ' nurse',
  'sequence': "I'm going to the hospital and meet a nurse"},
 {'score': 0.07787670195102692,
  'token': 27321,
  'token_str': ' psychiatrist',
  'sequence': "I'm going to the hospital and meet a psychiatrist"},
 {'score': 0.06631723791360855,
  'token': 16308,
  'token_str': ' surgeon',
  'sequence': "I'm going to the hospital and meet a surgeon"},
 {'score': 0.05966528132557869,
  'token': 1441,
  'token_str': ' friend',
  'sequence': "I'm going to the hospital and meet a friend"}]

In [35]:
# unmasker("병원에 가서 <mask>를 보고 올 거에요.") 안 먹히네

In [39]:
unmasker("If you have a <mask>, why don't we go talk about it somewhere only we know.")

[{'score': 0.34329357743263245,
  'token': 936,
  'token_str': ' problem',
  'sequence': "If you have a problem, why don't we go talk about it somewhere only we know."},
 {'score': 0.028329959139227867,
  'token': 864,
  'token_str': ' question',
  'sequence': "If you have a question, why don't we go talk about it somewhere only we know."},
 {'score': 0.027582349255681038,
  'token': 11096,
  'token_str': ' disability',
  'sequence': "If you have a disability, why don't we go talk about it somewhere only we know."},
 {'score': 0.021329136565327644,
  'token': 1928,
  'token_str': ' baby',
  'sequence': "If you have a baby, why don't we go talk about it somewhere only we know."},
 {'score': 0.017204632982611656,
  'token': 527,
  'token_str': ' story',
  'sequence': "If you have a story, why don't we go talk about it somewhere only we know."}]

In [40]:
unmasker = pipeline(task="fill-mask", model="google-bert/bert-base-uncased")
unmasker("Hello, I'm a [MASK] model.")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'score': 0.1441437155008316,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello, i ' m a role model."},
 {'score': 0.14175789058208466,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello, i ' m a fashion model."},
 {'score': 0.062214579433202744,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello, i ' m a new model."},
 {'score': 0.041028350591659546,
  'token': 3565,
  'token_str': 'super',
  'sequence': "hello, i ' m a super model."},
 {'score': 0.025911200791597366,
  'token': 2449,
  'token_str': 'business',
  'sequence': "hello, i ' m a business model."}]

## 📌InferenceAPI 사용

In [42]:
from dotenv import load_dotenv
import os
load_dotenv()
# 허깅페이스 토큰 READ 권한으로 생성하여 .env에 추가 (.gitignore 파일 추가)

True